In [1]:
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-6c59  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  5 06:32 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
bucket_name = 'que_buck' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_gcp/0_000.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_posting_locs.pickle
postings_gcp/101_000.bin
postings_gcp/101_posting_locs.pickle
postings_gcp/102_000.bin
postings_gcp/102_posting_locs.pickle
postings_gcp/103_000.bin
postings_gcp/103_posting_locs.pickle
postings_gcp/104_000.bin
postings_gcp/104_posting_locs.pickle
postings_gcp/105_000.bin
postings_gcp/105_posting_locs.pickle
postings_gcp/106_000.bin
postings_gcp/106_posting_locs.pickle
postings_gcp/107_000.bin
postings_gcp/107_posting_locs.pickle
postings_gcp/108_000.bin
postings_gcp/108_posting_locs.pickle
postings_gcp/109_000.bin
postings_gcp/109_posting_locs.pickle
postings_gcp/10_000.bin
postings_gcp/10_posting_locs.pickle
postings_gcp/110_000.bin
postings_gcp/110_posting_locs.pickle
postings_gcp/111_000.bin
postings_gcp/111_posting_locs.pickle
postings_gcp/112_000.bin
postings_gcp/112_posting_locs.pickle
postings_gcp/113_000.bin
postings_gcp/113_posting_locs.pickle

In [7]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
import json

doc_anchor_pairs = parquetFile.select("anchor_text","id").rdd




In [8]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [9]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [10]:
from inverted_index_gcp import InvertedIndex

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  d={}
  for token in tokens:
    if(token not in all_stopwords):
      if(token not in d.keys()):
        d[token]=(id,1)
      else:
        d[token]=(id,d[token][1]+1)
  return list(zip(d.keys(),d.values()))


def reduce_word_counts(unsorted_pl):
  lst=list(unsorted_pl)
  lst.sort(key=lambda x:x[0][0])
  return lst

def reduce_word_counts1(unsorted_pl):
  lst=list(unsorted_pl)
  lst.sort(key=lambda x:x[0])
  return lst

def calculate_df(postings):
 return postings.map(lambda x: (x[0], len(x[1])))

def calculate_total_term(word,pl):
  total=0
  for _,tf in pl:
    total+=tf
  return word,total

def get_pagerank(postings):
  vertices=[]
  edges=[]
  for docs_id in postings:
      edges.append(docs_id)
      vertices.append((docs_id[0],))
      vertices.append((docs_id[1],))
  edges=set(edges)
  vertices=list(set(vertices))
  v=sc.parallelize(vertices).toDF(['id'])
  e=sc.parallelize(edges).toDF(['src', 'dst'])
  g = GraphFrame(v, e)
  pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
  pr = pr_results.vertices.select("id","pagerank")
  res=pr.rdd.map(lambda x: (x[0],x[1])).collect()
  return res

def union_count(lst):
  dic={}
  for docs,value in lst:
    if docs in dic.keys():
      dic[docs]+=value
    else:
      dic[docs]=value
  return list(zip(dic.keys(),dic.values()))

def calculate_len(pairs):
  return pairs.map(lambda x: (x[0], len(x[1])))

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  l=postings.groupBy(lambda x: token2bucket_id(x[0])+248)
  return l.map(lambda y:InvertedIndex.write_a_posting_list((y[0],y[1]),bucket_name))

In [12]:
words = doc_anchor_pairs.flatMap(lambda y: map(lambda x: word_count(x[1],(y[1],x[0])),y[0]))
word_counts=words.flatMap(lambda x:x)
wordsone=word_counts.mapValues(lambda y:(y[0][1],y[1]))

postings=wordsone.groupByKey().mapValues(lambda x:union_count(sorted(list(x),key=lambda x:x[0])))
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(postings_filtered).collect()

In [13]:
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

Putting it all together

In [17]:
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
totals=postings.map(lambda x:calculate_total_term(x[0],x[1]))
inverted.term_total=totals.collectAsMap()

inverted.write_index('.', 'index_anchor')
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_anchor.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 33.9 MiB/ 33.9 MiB]                                                
Operation completed over 1 objects/33.9 MiB.                                     
